# <div align="center" style="color: #ff5733;">New Pos Addition and Mapping</div>

In [1]:
# %% [markdown]
# # Jupyter Notebook Loading Header
#
# This is a custom loading header for Jupyter Notebooks in Visual Studio Code.
# It includes common imports and settings to get you started quickly.

# %% [markdown]
## Import Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from google.cloud import bigquery
import os
path = r'C:\Users\DwaipayanChakroborti\AppData\Roaming\gcloud\legacy_credentials\dchakroborti@tonikbank.com\adc.json'
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = path
client = bigquery.Client(project='prj-prod-dataplatform')

# %% [markdown]
## Configure Settings
# Set options or configurations as needed
# Example: pd.set_option('display.max_columns', None)

In [2]:
df = pd.read_csv(r"C:\Users\DwaipayanChakroborti\OneDrive - Tonik Financial Pte Ltd\MyStuff\SIL Sales\Shin Nebria\Paula Denice Garibay\Pos_addition_20240628.csv")

In [ ]:
df = pd.read_excel(r"C:\Users\DwaipayanChakroborti\OneDrive - Tonik Financial Pte Ltd\MyStuff\SIL Sales\Shin Nebria\POS_Addition\Data\POS_mapping_data May042024.xlsx", sheet_name = 'Sheet1')
df.head(10)

In [3]:
df.head()

,POS NAME,STORE LOCATION,STATUS,DATE ACTIVATION,PURPLE KEY,REGION,TSM,Brand
0,Regie C Basillo Lotus Mall Imus,Lotus Mall Imus,POS Operational,"June 28, 2024",M01FEG9,South Luzon 1 (B),Ricky Sarayno,RCB Mobile
1,RCB Multibrand Lotus Mall,Lotus Mall Imus,POS Operational,"June 28, 2024",M39HE4Y,South Luzon 1 (B),Ricky Sarayno,RCB Mobile
2,K Multibrand Binakayan,Standalone,POS Operational,"June 28, 2024",MKGZCLU,South Luzon 1 (B),Ricky Sarayno,RCB Mobile


In [4]:
df.rename(columns = {'Brand': 'BrandStores'}, inplace = True)
df.columns

Index(['POS NAME', 'STORE LOCATION', 'STATUS', 'DATE ACTIVATION', 'PURPLE KEY',
       'REGION', 'TSM', 'BrandStores'],
      dtype='object')

In [5]:
df['BrandStores'].value_counts()

BrandStores
RCB Mobile    3
Name: count, dtype: int64

In [6]:
df['PURPLE KEY'].value_counts()

PURPLE KEY
M01FEG9    1
M39HE4Y    1
MKGZCLU    1
Name: count, dtype: int64

In [7]:
df = df[['BrandStores', 'POS NAME', 'STORE LOCATION', 'STATUS', 'DATE ACTIVATION', 'PURPLE KEY', 'REGION', 'TSM']]
df

,BrandStores,POS NAME,STORE LOCATION,STATUS,DATE ACTIVATION,PURPLE KEY,REGION,TSM
0,RCB Mobile,Regie C Basillo Lotus Mall Imus,Lotus Mall Imus,POS Operational,"June 28, 2024",M01FEG9,South Luzon 1 (B),Ricky Sarayno
1,RCB Mobile,RCB Multibrand Lotus Mall,Lotus Mall Imus,POS Operational,"June 28, 2024",M39HE4Y,South Luzon 1 (B),Ricky Sarayno
2,RCB Mobile,K Multibrand Binakayan,Standalone,POS Operational,"June 28, 2024",MKGZCLU,South Luzon 1 (B),Ricky Sarayno


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3 entries, 0 to 2
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   BrandStores      3 non-null      object
 1   POS NAME         3 non-null      object
 2   STORE LOCATION   3 non-null      object
 3   STATUS           3 non-null      object
 4   DATE ACTIVATION  3 non-null      object
 5   PURPLE KEY       3 non-null      object
 6   REGION           3 non-null      object
 7   TSM              3 non-null      object
dtypes: object(8)
memory usage: 320.0+ bytes


In [9]:
df.rename(columns={'POS NAME':'POS', 'DATE ACTIVATION':'DateOpened', 'STORE LOCATION': 'StoreLocation', 'STATUS':'Status', 'PURPLE KEY':'PurpleKey', 'REGION':'Region'}, inplace = True)

In [10]:
import pandas as pd

df['DateOpened'] = pd.to_datetime(df['DateOpened'], format='%B %d, %Y')


In [11]:
print(df['DateOpened'].dtype)
# datetime64[ns]
print(df['DateOpened'].iloc[0]) 
# 2024-01-15 00:00:00
df['DateOpened'] = df['DateOpened'].dt.strftime('%Y-%m-%d')
df.info()

datetime64[ns]
2024-06-28 00:00:00
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3 entries, 0 to 2
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   BrandStores    3 non-null      object
 1   POS            3 non-null      object
 2   StoreLocation  3 non-null      object
 3   Status         3 non-null      object
 4   DateOpened     3 non-null      object
 5   PurpleKey      3 non-null      object
 6   Region         3 non-null      object
 7   TSM            3 non-null      object
dtypes: object(8)
memory usage: 320.0+ bytes


In [12]:
import pandas as pd

df['DateOpened'] = pd.to_datetime(df['DateOpened'], format='%d-%b-%y')

ValueError: time data "2024-06-28" doesn't match format "%d-%b-%y", at position 0. You might want to try:
    - passing `format` if your strings have a consistent format;
    - passing `format='ISO8601'` if your strings are all ISO8601 but not necessarily in exactly the same format;
    - passing `format='mixed'`, and the format will be inferred for each element individually. You might want to use `dayfirst` alongside this.

In [ ]:
print(df['DateOpened'].dtype)
# datetime64[ns]

object


In [13]:
print(df['DateOpened'].iloc[0]) 
# 2024-01-15 00:00:00

2024-06-28


In [14]:
df['DateOpened'] = df['DateOpened'].dt.strftime('%Y-%m-%d')
df.info()

AttributeError: Can only use .dt accessor with datetimelike values

In [15]:
df['DateOpened'] = pd.to_datetime(df['DateOpened'], format='%Y-%m-%d')

In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3 entries, 0 to 2
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   BrandStores    3 non-null      object        
 1   POS            3 non-null      object        
 2   StoreLocation  3 non-null      object        
 3   Status         3 non-null      object        
 4   DateOpened     3 non-null      datetime64[ns]
 5   PurpleKey      3 non-null      object        
 6   Region         3 non-null      object        
 7   TSM            3 non-null      object        
dtypes: datetime64[ns](1), object(7)
memory usage: 320.0+ bytes


In [17]:
df.head()

,BrandStores,POS,StoreLocation,Status,DateOpened,PurpleKey,Region,TSM
0,RCB Mobile,Regie C Basillo Lotus Mall Imus,Lotus Mall Imus,POS Operational,2024-06-28,M01FEG9,South Luzon 1 (B),Ricky Sarayno
1,RCB Mobile,RCB Multibrand Lotus Mall,Lotus Mall Imus,POS Operational,2024-06-28,M39HE4Y,South Luzon 1 (B),Ricky Sarayno
2,RCB Mobile,K Multibrand Binakayan,Standalone,POS Operational,2024-06-28,MKGZCLU,South Luzon 1 (B),Ricky Sarayno


In [18]:
sq = """drop table if exists prj-prod-dataplatform.dap_ds_poweruser_playground.POS_TSM_MAPIING1;"""
client.query(sq)
sq = """select * from prj-prod-dataplatform.dap_ds_poweruser_playground.POS_TSM_MAPIING1;"""
try:
    d = client.query(sq).to_dataframe(progress_bar_type='tqdm')
    print("Table still exists")
except:
    print("Table dropped")

Query is running:   0%|          |Table dropped


In [19]:
sq = """create table prj-prod-dataplatform.dap_ds_poweruser_playground.POS_TSM_MAPIING1
as 
select * from prj-prod-dataplatform.dap_ds_poweruser_playground.POS_TSM_MAPIING;
"""
client.query(sq)



QueryJob<project=prj-prod-dataplatform, location=asia-southeast1, id=597c9e37-bae5-4662-ba68-67790fd7fa7f>

In [20]:
sq = """select * from prj-prod-dataplatform.dap_ds_poweruser_playground.POS_TSM_MAPIING1;"""
d = client.query(sq).to_dataframe(progress_bar_type='tqdm')
print(f"The shape of d is:\t{d.shape}")

Job ID ae5d9254-8f73-494a-9f34-0345ec707f68 successfully executed: 100%|██████████|
Downloading: 100%|██████████|
The shape of d is:	(675, 8)


In [21]:
sq = """select * from prj-prod-dataplatform.dap_ds_poweruser_playground.POS_TSM_MAPIING1"""

dfsil = client.query(sq).to_dataframe(progress_bar_type = 'tqdm')

Job ID b1e61083-871c-4c94-8975-e21ac0659aae successfully executed: 100%|██████████|
Downloading: 100%|██████████|


In [22]:
dfsil.BrandStores.value_counts().sort_index()

BrandStores
A2RJ                  3
Affordables           2
Ansons               15
ArkHome               4
Automatic Centre     11
Dizon Furniture       1
ESLA                 51
FC Home             109
Faith Glorious        9
Gadgetmate           10
Hardware Sugar        2
Home Along           91
Home Factory          2
Homeworks            13
JSL                   6
JTG                   1
Jhunpyo               8
Junliz                8
KUM Home Center       3
Kservico             10
MemoXpress           54
PHPS                  2
Power Mac Center      5
Presnet              21
Prince and Aviel      3
Rosa Mobile           9
Rulls               161
Sogo Home             3
Willy and Sons       29
Workstation           6
Wyse                  1
Xtreme               22
Name: count, dtype: int64

In [23]:
dfsil.shape

(675, 8)

In [24]:
# Set table_id to the ID of the table to append to.
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.POS_TSM_MAPIING1"

job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND"
)

job = client.load_table_from_dataframe(
    df, table_id, job_config=job_config
)  

job.result()  # Wait for the job to complete.

table = client.get_table(table_id)  # Make an API request.
print(
    "Loaded {} rows and {} columns to {}".format(
        table.num_rows, len(table.schema), table_id
    )
)

Loaded 678 rows and 8 columns to prj-prod-dataplatform.dap_ds_poweruser_playground.POS_TSM_MAPIING1


In [25]:
sq = """select * from prj-prod-dataplatform.dap_ds_poweruser_playground.POS_TSM_MAPIING1"""
df = client.query(sq).to_dataframe(progress_bar_type = 'tqdm')
print(f"The shape of the dataframe is: {df.shape}")

Job ID b380b9e5-e2f5-41bf-9c0e-088079296e77 successfully executed: 100%|██████████|
Downloading: 100%|██████████|
The shape of the dataframe is: (678, 8)


In [26]:
df.tail(10)

,BrandStores,POS,StoreLocation,Status,DateOpened,PurpleKey,Region,TSM
668,FC Home,FC Home Center Mamatid,Standalone,POS Operational,2023-09-14,MC31AV7,South Luzon 2,Christian Klide Alberto
669,FC Home,FC Home Center Cabuyao 1,Standalone,POS Operational,2023-09-14,M5D8JOH,South Luzon 2,Christian Klide Alberto
670,FC Home,FC Home Center Binan,Central Mall Laguna,POS Operational,2023-09-14,MYRK3EJ,South Luzon 2,Christian Klide Alberto
671,FC Home,FC Home Center Sta Cruz,Standalone,POS Operational,2023-09-14,M1ZYHKU,South Luzon 2,Christian Klide Alberto
672,FC Home,FC Home Center San Pablo,Standalone,POS Operational,2023-09-14,MADP0JZ,South Luzon 2,Christian Klide Alberto
673,FC Home,FC Home Center Siniloan,Standalone,POS Operational,2023-09-15,MANB6XQ,South Luzon 2,Christian Klide Alberto
674,Workstation,Workstation Home Appliances Target Mall,Target Mall,POS Operational,2023-10-13,M8I4QGT,South Luzon 2,Christian Klide Alberto
675,Workstation,Workstation Home Appliances Central Mall,Central Mall,POS Operational,2023-10-13,MTNCOR5,South Luzon 2,Christian Klide Alberto
676,Workstation,Workstation Home Appliances Pavilion Mall 1,Pavillon Mall,POS Operational,2023-10-13,MN8MWPJ,South Luzon 2,Christian Klide Alberto
677,Xtreme,Xtreme Appliances Calamba,Carmel Mall,POS Operational,2023-10-15,MUBPW04,South Luzon 2,Christian Klide Alberto


In [27]:
df['BrandStores'].value_counts().sort_index()

BrandStores
A2RJ                  3
Affordables           2
Ansons               15
ArkHome               4
Automatic Centre     11
Dizon Furniture       1
ESLA                 51
FC Home             109
Faith Glorious        9
Gadgetmate           10
Hardware Sugar        2
Home Along           91
Home Factory          2
Homeworks            13
JSL                   6
JTG                   1
Jhunpyo               8
Junliz                8
KUM Home Center       3
Kservico             10
MemoXpress           54
PHPS                  2
Power Mac Center      5
Presnet              21
Prince and Aviel      3
RCB Mobile            3
Rosa Mobile           9
Rulls               161
Sogo Home             3
Willy and Sons       29
Workstation           6
Wyse                  1
Xtreme               22
Name: count, dtype: int64

In [28]:
# Find duplicate values in 'POS' column
duplicates = df[df.duplicated('POS', keep=False)]

# Print the entire rows for duplicate entries in 'POS'
if not duplicates.empty:
    print("Duplicate values in 'POS' column:")
    duplicates.to_csv("Temp.csv")
else:
    print("No duplicate values found in 'POS' column.")
    
dd = pd.read_csv("Temp.csv")
dd.sort_values(by='POS')

Duplicate values in 'POS' column:


,Unnamed: 0,BrandStores,POS,StoreLocation,Status,DateOpened,PurpleKey,Region,TSM
0,5,Hardware Sugar,Hardware Sugar Makati,NaN,NaN,2022-07-04,MV4UO0X,NaN,NaN
1,6,Hardware Sugar,Hardware Sugar Makati,NaN,NaN,2022-07-04,M9ZK6IU,NaN,NaN
12,637,Home Along,Home Along - Mayon,Standalone,POS Operational,2022-07-01,MN3GL66,NCR North 2,Ben Antonio Lacsamana
13,638,Home Along,Home Along - Mayon,Standalone,POS Operational,2022-07-01,MGP2AQI,NCR North,Ben Antonio Lacsamana
14,647,Home Along,Home Along Malanday,NaN,NaN,2022-07-11,MURFAIV,NCR North,Ben Antonio Lacsamana
15,648,Home Along,Home Along Malanday,Standalone,POS Operational,2022-07-11,MNBW4RZ,NCR North,Ben Antonio Lacsamana
4,560,Home Along,Home Along Muntinlupa,Standalone,POS Operational,2022-03-26,MDGYHQJ,NCR South,Klein Justine Tamayo
5,562,Home Along,Home Along Muntinlupa,Standalone,POS Operational,2022-03-26,MU82X9Z,NCR South,Klein Justine Tamayo
6,574,Home Along,Home Along Puregold Taguig,Standalone,POS Operational,2022-07-01,MRGP0WQ,NCR South,Klein Justine Tamayo
10,578,Home Along,Home Along Puregold Taguig,Standalone,POS Operational,2022-07-01,MDO3KI4,NCR South,Klein Justine Tamayo


In [29]:
pd.set_option('display.max_rows', None)
# Count the occurrences of each value in the 'PurpleKey' column
purple_key_counts = df['PurpleKey'].value_counts()

# Filter to include only values with count >= 2
filtered_counts = purple_key_counts[purple_key_counts >= 2]

# Display the filtered counts
print(filtered_counts)


Series([], Name: count, dtype: int64)


In [30]:
df[df['PurpleKey']== 'M01FEG9']

,BrandStores,POS,StoreLocation,Status,DateOpened,PurpleKey,Region,TSM
0,RCB Mobile,Regie C Basillo Lotus Mall Imus,Lotus Mall Imus,POS Operational,2024-06-28,M01FEG9,South Luzon 1 (B),Ricky Sarayno


In [31]:
sq = """drop table prj-prod-dataplatform.dap_ds_poweruser_playground.POS_TSM_MAPIING;"""

client.query(sq)


QueryJob<project=prj-prod-dataplatform, location=asia-southeast1, id=4f4fea05-2d63-4c76-9bac-82b101d643f3>

In [32]:
sq = """create table prj-prod-dataplatform.dap_ds_poweruser_playground.POS_TSM_MAPIING as select * from prj-prod-dataplatform.dap_ds_poweruser_playground.POS_TSM_MAPIING1;"""
client.query(sq)


QueryJob<project=prj-prod-dataplatform, location=asia-southeast1, id=c30f17d7-f3c4-4550-880a-d4132950acfb>

In [33]:
client.query("commit")

QueryJob<project=prj-prod-dataplatform, location=US, id=cecd8ac2-8ed1-453f-946d-31121b1021f1>

In [34]:
sq = """select * from prj-prod-dataplatform.dap_ds_poweruser_playground.POS_TSM_MAPIING;"""

df = client.query(sq).to_dataframe(progress_bar_type='tqdm')
df.shape

Job ID e6f7b460-b984-44b0-b274-7f17ac423b44 successfully executed: 100%|██████████|
Downloading: 100%|██████████|


(678, 8)

In [36]:
df[df['BrandStores']=='RCB Mobile'].sort_values(by='DateOpened')

,BrandStores,POS,StoreLocation,Status,DateOpened,PurpleKey,Region,TSM
116,RCB Mobile,RCB Multibrand Lotus Mall,Lotus Mall Imus,POS Operational,2024-06-28,M39HE4Y,South Luzon 1 (B),Ricky Sarayno
117,RCB Mobile,Regie C Basillo Lotus Mall Imus,Lotus Mall Imus,POS Operational,2024-06-28,M01FEG9,South Luzon 1 (B),Ricky Sarayno
118,RCB Mobile,K Multibrand Binakayan,Standalone,POS Operational,2024-06-28,MKGZCLU,South Luzon 1 (B),Ricky Sarayno


In [37]:
df.head(10)

,BrandStores,POS,StoreLocation,Status,DateOpened,PurpleKey,Region,TSM
0,FC Home,FC Home Center Balibago,LOT 5 BLK 9 GARNET ST. RHINELAND COMMERCIAL CO...,Operational,2023-08-01,MOG243Y,South Luzon 2,Klide Alberto
1,Xtreme,Xtreme Appliances Anabu,None,None,2022-08-27,MR2YDE8,None,None
2,Hardware Sugar,Hardware Sugar Makati,None,None,2022-07-04,MV4UO0X,None,None
3,Hardware Sugar,Hardware Sugar Makati,None,None,2022-07-04,M9ZK6IU,None,None
4,Home Along,Home Along Sumulong,None,None,2022-07-12,M2G78K4,None,None
5,Dizon Furniture,Dizon Furniture Tiendesitas,None,None,2022-07-23,MOE3JQH,None,None
6,Xtreme,Xtreme Appliances Lucky Gold Plaza,Lucky Gold Plaza,For Opening,2024-04-14,MO3AHGW,NCR East,Albee David
7,Rulls,MBC Gaisano Inline Binangonan,Gaisano Inline Binangonan,For Opening,2024-04-15,M38S4QA,NCR East,Albee David
8,Rulls,Mobile Craze Town Center Mall Tanay,Craze Town Center Mall Tanay,For Opening,2024-04-15,M3ED6SF,NCR East,Albee David
9,Rulls,Mobilecraze K2 PG Marikina,Puregold Marikina,For Opening,2024-04-15,MUFDX4A,NCR East,Albee David


In [38]:
df.sort_values('DateOpened', ascending=False)

,BrandStores,POS,StoreLocation,Status,DateOpened,PurpleKey,Region,TSM
116,RCB Mobile,RCB Multibrand Lotus Mall,Lotus Mall Imus,POS Operational,2024-06-28,M39HE4Y,South Luzon 1 (B),Ricky Sarayno
117,RCB Mobile,Regie C Basillo Lotus Mall Imus,Lotus Mall Imus,POS Operational,2024-06-28,M01FEG9,South Luzon 1 (B),Ricky Sarayno
118,RCB Mobile,K Multibrand Binakayan,Standalone,POS Operational,2024-06-28,MKGZCLU,South Luzon 1 (B),Ricky Sarayno
247,JSL,AS Sabayo Wilcon Visayas Ave,Standalone,For Opening,2024-06-26,MYPG7T9,NCR North,Sander Maghirang
245,JSL,AS Sabayo Robinsons Novaliches,Robinsons Novaliches,For Opening,2024-06-26,MZITK84,NCR North,Sander Maghirang
248,JSL,JSL Waltermart Munoz,Waltermart Munoz,For Opening,2024-06-26,M7P1GIK,NCR North,Sander Maghirang
244,JSL,ABLNR iMall,iMall Caloocan,For Opening,2024-06-26,MC08YM1,NCR North,Sander Maghirang
243,JSL,SHAR Novaliches,Standalone,For Opening,2024-06-26,MDZSR97,NCR North,Sander Maghirang
246,JSL,JSL Novaliches,Standalone,For Opening,2024-06-26,M65WJCU,NCR North,Sander Maghirang
345,Presnet,Samsung Rosario,Standalone,For Opening,2024-06-25,M6O85IU,South Luzon 3,Jan jan Dela Cruz


In [39]:
df[df['PurpleKey'] =='M01FEG9']

,BrandStores,POS,StoreLocation,Status,DateOpened,PurpleKey,Region,TSM
117,RCB Mobile,Regie C Basillo Lotus Mall Imus,Lotus Mall Imus,POS Operational,2024-06-28,M01FEG9,South Luzon 1 (B),Ricky Sarayno


In [40]:
df[df['BrandStores']=='RCB Mobile']

,BrandStores,POS,StoreLocation,Status,DateOpened,PurpleKey,Region,TSM
116,RCB Mobile,RCB Multibrand Lotus Mall,Lotus Mall Imus,POS Operational,2024-06-28,M39HE4Y,South Luzon 1 (B),Ricky Sarayno
117,RCB Mobile,Regie C Basillo Lotus Mall Imus,Lotus Mall Imus,POS Operational,2024-06-28,M01FEG9,South Luzon 1 (B),Ricky Sarayno
118,RCB Mobile,K Multibrand Binakayan,Standalone,POS Operational,2024-06-28,MKGZCLU,South Luzon 1 (B),Ricky Sarayno


In [41]:
df.to_excel(r"C:\Users\DwaipayanChakroborti\OneDrive - Tonik Financial Pte Ltd\MyStuff\SIL Sales\Shin Nebria\POS_Addition\Data\POS_mapping_data.xlsx", index = False)